<center>
<img src='./img/nsidc_logo.png'/>

# **IceFlow**
### Point Cloud Data Access
</center>

---

## Visualizing large datasets
Valkyrie and Icesat 2 data are by nature big data sets that require some special considerations when working with it. The main constraint if you don't have a super computer is memory. The average granule size is in the 10s of MB for IceSat 2 and could be Gigabytes in Valkyrie depending on the order/subsetting. 

This is when libraries like Dask, Vaex and others come into play. This notebook will show you how to use some basic plotting techniques using matpplotlib and geopandas + Vaex to work effectively with lidar data from Valkyrie and ATL data from IceSat 2.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import glob
import geopandas
import pandas as pd
import h5py
import vaex
import dask.dataframe as dd
import dask.array as da
import numpy as np
from iceflow.processing import IceFlowProcessing as ifp

# filepath = 'data/atm1b_data_2020-07-10T15-32.hdf5'
# df_k = ifp.get_common_dictionary('ATM')

filepath = 'data/twaties-test-GLAH06-2000-2010.h5'
df_key = ifp.get_common_dictionary('GLASS')

## Loading the data with H5PY

In [ ]:
%%time

f = h5py.File(filepath, 'r')
print(list(f.keys()))

df = ifp.get_common_df(f)
display(df.describe())
df

## Vaex 

* Using VAEX to visualize big data frames.


In [ ]:
%%time

df = vaex.open(filepath)
# We're parsing the utc_datetime from Valkyrie into a data type that Vaex understands.
df['date'] = df.utc_datetime.values.astype('datetime64[ns]')
# my_df = df['longitude', 'latitude', 'elevation', 'date']
# Note that we need a common dic because in GLAH06 levatio is d_elev and in ICESat 2 is called elevation! 
my_df = df[df_key['latitude'], df_key['longitude'], df_key['elevation'], 'date']
# vaex.vrange() is like numpy.arange but uses 0-memory no matter the length.
# This is to downsample the data for dataviz see: https://github.com/vaexio/vaex/issues/911
df.add_column('index', vaex.vrange(0, len(df)))
# We are going to create a "decimated" dataframe with only 1/100 of the size of the original to plot the big picture faster.
df_decimated = df[(df.index % 100 == 0)]
my_df.describe()
display(my_df)

### Visualizing the big picture

In [ ]:
my_df.widget.heatmap(my_df[df_key['longitude']], 
               my_df[df_key['latitude']],
               what=vaex.stat.mean(my_df[df_key['elevation']]),
               shape=512, 
               figsize=(10,6),
               limits='minmax',
               colormap='inferno')

In [ ]:
%matplotlib widget
import vaex
from ipywidgets import widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

plt.figure(figsize=(10,8), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0)) 
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, -65, -90], ccrs.PlateCarree())
plt.scatter(df_decimated[df_key['longitude']].values,
            df_decimated[df_key['latitude']].values,
            c=df_decimated[df_key['elevation']].values,
            cmap='viridis',
            vmin=100,vmax=1000,
            transform=ccrs.PlateCarree())
plt.colorbar(label='elevation', shrink=0.5, extend='both')

## "Flying" with the sensor to get a closer look on the data

In [ ]:
%matplotlib widget
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(70, 70)

#If the data granule(s) is big enough (1+GB), use the decimated dataframe.
# df = df_decimated

def plot_func(alontrack):
    step = 5000 # same as density
    m = int(alontrack * step)
    ax.clear()
    ax.scatter(df[df_key['longitude']].values[m:m+step],
               df[df_key['latitude']].values[m:m+step],
               df[df_key['elevation']].values[m:m+step],
               c=df[df_key['elevation']].values[m:m+step],
               cmap='viridis', s=1)
    ax.axis('tight')
    
    

interact(plot_func, alontrack = widgets.FloatSlider(value=0,
                                                    description='Along Track Steps',
                                                    min=0,
                                                    max=90,
                                                    step=0.3,
                                                    layout={'width': '100%'}))

## Time Series

After harmonizing data from Pre-IB, IS1, IceBridge, IS2 we should be able to grid and or concatenate the dataframes and start building real analyses.

Ideas: 

* Use Numba when possible
* Dask deployed in DAACs not AWS
* Extend VAEX ala GeoPandas, proposed name: GeoVAEX
* Explore https://github.com/davidbrochart/xarray_leaflet
* See if IRIS can be used here ALA Pangeo.
* **Pangeo All the Things!!!**


<img src='./img/unfinished_horse.jpg' width="200px">